In [29]:
# Семинар 12: спецификация модели, тесты на выбор функциональной формы

import numpy as np
import pandas as pd # чтение файлов
import seaborn as sns # построение графиков
import statsmodels.api as sm # тесты
import statsmodels.formula.api as smf
import statsmodels
import pandas as pd
import statsmodels.stats.outliers_influence as oi
import scipy.stats
import math
from scipy.stats import chi2

In [3]:
# Импортируем данные о квартирах в Москве
d = pd.read_excel('/Users/polinapogorelova/Desktop/dataflats.xlsx')
d

,A,B,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor
0,1,81.0,58,40,6.0,12.5,7,1,1,1
1,2,75.0,44,28,6.0,13.5,7,1,0,1
2,3,128.0,70,42,6.0,14.5,3,1,1,1
3,4,95.0,61,37,6.0,13.5,7,1,0,1
4,5,330.0,104,60,11.0,10.5,7,0,1,1
...,...,...,...,...,...,...,...,...,...,...
2035,2036,110.0,77,45,10.0,12.0,5,0,0,1
2036,2037,95.0,60,43,6.0,9.0,5,0,0,1
2037,2038,95.0,60,46,5.0,10.5,5,1,0,1
2038,2039,129.0,76,48,10.0,12.5,5,0,0,1


In [6]:
# Изменим название столбцов A и B на n (номер наблюдения) и price, соответственно
d.rename(columns={'A':'n', 'B': 'price'}, inplace=True)
# Удалим строки, в которых есть пропуск хотя бы для одного столбца
d.dropna(inplace=True)
# Добавим в набор данных новую переменную price_sq (стоимость 1 кв м квартиры в Москве)
d['price_sq'] = d['price']/d['totsp']
d

,n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,price_sq
0,1,81.0,58,40,6.0,12.5,7,1,1,1,1.396552
1,2,75.0,44,28,6.0,13.5,7,1,0,1,1.704545
2,3,128.0,70,42,6.0,14.5,3,1,1,1,1.828571
3,4,95.0,61,37,6.0,13.5,7,1,0,1,1.557377
4,5,330.0,104,60,11.0,10.5,7,0,1,1,3.173077
...,...,...,...,...,...,...,...,...,...,...,...
2035,2036,110.0,77,45,10.0,12.0,5,0,0,1,1.428571
2036,2037,95.0,60,43,6.0,9.0,5,0,0,1,1.583333
2037,2038,95.0,60,46,5.0,10.5,5,1,0,1,1.583333
2038,2039,129.0,76,48,10.0,12.5,5,0,0,1,1.697368


In [21]:
# 1. Тест Рамсея для модели model
model = smf.ols('price_sq ~ totsp + livesp + kitsp + dist + metrdist', data=d).fit()
# H0: нет пропущенных переменных
# H1: есть пропущенные переменные
print(oi.reset_ramsey (model, degree = 2))

# Так как. p-value = 0, гипотеза H1 не отвергается, то есть в модели есть пропущенные переменные

<F test: F=array([[91.84997418]]), p=2.619541097736929e-21, df_denom=2.03e+03, df_num=1>


In [40]:
# 2. J-тест для невложенных моделей
model_1 = smf.ols('price_sq ~ livesp + kitsp + dist + metrdist + walk + brick', data = d).fit()
model_2 = smf.ols('price_sq ~ totsp + livesp + kitsp + dist + metrdist + brick', data = d).fit()
print(statsmodels.stats.diagnostic.compare_j(model_2, model_1))
print(statsmodels.stats.diagnostic.compare_j(model_1, model_2))

(2.047008449318996, 0.040785759407924616)
(8.146433320246322, 6.4759273373675e-16)


Рассмотрим PE-тест для сравнения линейной и полулогарифмической моделей:

1) Оцениваем линейную и полулогарифмическую (или логарифмическую) модель регрессии:
    $$Y = X \beta + \varepsilon,$$
    $$\log Y = X \beta + \varepsilon.$$

2) Оценим вспомогательные регрессии $$Y = X \beta + \delta_{LIN} \left(\log \hat{Y} - \widehat{\log Y} \right) + \varepsilon,$$
$$\log Y = X \beta + \delta_{LOG} \left(\hat{Y} - \exp (\widehat{\log Y}) \right) + \varepsilon.$$

3) Проверим гипотезы $H_0$: $\delta_{LIN} = 0$ и $H_0'$: $\delta_{LOG} = 0$.

In [88]:
# 3. PE-тест Дэвидсона-Маккинона (выбор между линейной и полулогарифмической (или линейной в логарифмах) моделями
d['ln_price_sq'] = np.log(d['price_sq'])
model_lin = smf.ols('price_sq ~ totsp + livesp + kitsp + dist + metrdist', data = d).fit()
model_log = smf.ols('ln_price_sq ~ totsp + livesp + kitsp + dist + metrdist', data = d).fit()

# Вспомогательные регрессии
d['lin'] = np.log(model_lin.fittedvalues) - model_log.fittedvalues
d['log'] = model_lin.fittedvalues - np.exp(model_log.fittedvalues)
model_lin_add = smf.ols('price_sq ~ totsp + livesp + kitsp + dist + metrdist + lin', data = d).fit()
model_log_add = smf.ols('ln_price_sq ~ totsp + livesp + kitsp + dist + metrdist + log', data = d).fit()
print(model_lin_add.summary())
print(model_log_add.summary())

                            OLS Regression Results                            
Dep. Variable:               price_sq   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     179.1
Date:                Sun, 10 Dec 2023   Prob (F-statistic):          3.24e-183
Time:                        23:31:12   Log-Likelihood:                -618.01
No. Observations:                2038   AIC:                             1250.
Df Residuals:                    2031   BIC:                             1289.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4133      0.063     22.518      0.0

In [ ]:
# 4. Тест Бокса-Кокса: выбор между линейной и линейной в логарифмах моделями
# H0: lambda = 0 (полулогарифмическая)
# H1: lambda != 0 (неполулогарифмическая)

# H0: lambda = 1 (линейная)
# H1: lambda != 1 (нелинейная)

In [70]:
# 5. Метод Зарембки (частный случай преобразования Бокса-Кокса): выбор между линейной и полулогарифмической моделями
# H0: качество подгонки линейной и полулогарифмической моделей одинаковое
# H1: модель с меньшей RSS лучше
g_price_sq = math.exp(np.mean(np.log(d['price_sq']))) # геометрическое среднее
print("Геометрическое среднее =", g_price_sq)
g_price_sq # геометрическое среднее (автоматически)

d['y_new'] = d['price_sq']/g_price_sq
d['ln_y_new'] = np.log(d['y_new'])

model1 = smf.ols('y_new ~ totsp + livesp + kitsp + dist + metrdist', data=d).fit()
model2 = smf.ols('ln_y_new ~ totsp + livesp + kitsp + dist + metrdist', data=d).fit()

xi_2 = 2038/2*abs(np.log(model1.ssr/model2.ssr)) # значение тестовой статистики
print("Наблюдаемое значение статистики =", xi_2)
xi_crit = chi2.ppf(0.95, 1) # критическое значение
print("Критическое значение статистики =",xi_crit) 

# Так как наблюдаемое значение статистики больше критического, то на 5% уровне мы не можем отвергнуть гипотезу H1,
# то есть модели имеют разное качество подгонки

# Выбор модели производится бы на основе RSS (чем RSS меньше, тем лучше)
print("RSS для model_1 =", model1.ssr) # RSS для линейной модели
print("RSS для model_2 =", model2.ssr) # RSS для полулогарифмической модели (оказалась лучше)

Геометрическое среднее = 1.679712454737158
Наблюдаемое значение статистики = 333.01269443436007
Критическое значение статистики = 3.841458820694124
RSS для model_1 = 81.06498820522367
RSS для model_2 = 58.46613672464129


In [46]:
# 6. Тестирование нормальности остатков модели с помощью теста Харке-Бера
# H0: остатки имеют нормальное распределение
# H1: распределение остатков отлично от нормального
from scipy import stats
print(stats.jarque_bera(model1.resid)) # так как p-value > 0.05, то гипотеза H0 не отвергается на любом разумном уровне значимости,
# то есть остатки линейной модели можно считать нормальными

print(stats.jarque_bera(model2.resid)) # так как p-value > 0.05, то гипотеза H0 не отвергается на любом разумном уровне значимости,
# то есть остатки линейной модели можно считать нормальными

Jarque_beraResult(statistic=6948.918638162937, pvalue=0.0)
Jarque_beraResult(statistic=538.2320313063323, pvalue=0.0)
